In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from scipy.ndimage import convolve, generate_binary_structure

In [4]:
Nx = 25
Ny = 25
Nz = 70

init_random = np.random.random((Nx, Ny, Nz))
lattice = -np.ones((Nx, Ny, Nz)) #initialize all values with -1
#lattice[init_random >= 0.5] = -1    

In [3]:
def get_energy(lattice):
    # applies the nearest neighbours summation
    kern = generate_binary_structure(3, 1) 
    kern[1][1][1] = False
    print(kern)
    arr = -lattice * convolve(lattice, kern, mode='constant', cval=0)
    return arr.sum()

In [7]:
def metropolis(spin_arr, times, BJ):
    spin_arr = spin_arr.copy()
    for t in range(0,times-1):
        # 2. pick random point on array and flip spin
        x = np.random.randint(0,Nx)
        y = np.random.randint(0,Ny)
        z = np.random.randint(0,Nz)
        spin_i = spin_arr[x,y,z] #initial spin
        spin_f = spin_i*-1 #proposed spin flip
        
        # compute change in energy
        E_i = 0
        E_f = 0
        if x>0:
            E_i += -spin_i*spin_arr[x-1,y,z]
            E_f += -spin_f*spin_arr[x-1,y,z]
        if x<Nx-1:
            E_i += -spin_i*spin_arr[x+1,y,z]
            E_f += -spin_f*spin_arr[x+1,y,z]
        if y>0:
            E_i += -spin_i*spin_arr[x,y-1,z]
            E_f += -spin_f*spin_arr[x,y-1,z]
        if y<Ny-1:
            E_i += -spin_i*spin_arr[x,y+1,z]
            E_f += -spin_f*spin_arr[x,y+1,z]
        if z>0:
            E_i += -spin_i*spin_arr[x,y,z-1]
            E_f += -spin_f*spin_arr[x,y,z-1]
        if z<Nz-1:
            E_i += -spin_i*spin_arr[x,y,z+1]
            E_f += -spin_f*spin_arr[x,y,z+1]
        
        # 3 / 4. change state with designated probabilities
        dE = E_f-E_i
        if (dE>0)*(np.random.random() < np.exp(-BJ*dE)):
            spin_arr[x,y,z]=spin_f
        elif dE<=0:
            spin_arr[x,y,z]=spin_f
            
    return spin_arr

In [11]:
NuT = 10
Lc = 0.01
A = 1e-6
R = 1e-2
Hc = 1e3 #scaling
Mc=  1e5 #scaling

k = 1
mu_0 = 4*np.pi*1e-7

Time = 0.5e-4
dt = 0.2e-6
n = np.floor(Time/dt)
Ival = 3

sw_speed = 1e7 #number of switches in one second
mc_steps = sw_speed*dt

I = []
I.extend(Ival*np.ones(50))
I.extend(-Ival*np.ones(50))
I.extend(Ival*np.ones(50))
k=len(I)

VL = [0]*len(I)
IL = [0]*len(I)
IR = [0]*len(I)
V = [0]*len(I)
ILguess=[0,0]
phi = [0]*len(I)
L = [0]*len(I)
H = [0]*len(I)
B = [0]*len(I)
M = [0]*len(I)

t = np.linspace(0,dt*k,k)

Time = 2*n*dt


for i in range(1,3):

    ILguess[0]=I[i]*0.1 #first guess
    diff=1

    while(diff>1e-9):        
        H[i] = ILguess[0]*NuT/Lc

        lattice = metropolis(lattice, int(mc_steps), H[i]/Hc,)
        M[i] = lattice.sum()/(Nx*Ny*Nz)*Mc
        B[i] = mu_0*(H[i]+M[i])
        phi[i]=B[i]*A

        IR[i]= (phi[i]-phi[i-1])/dt/R
        ILguess[1]= I[i] - IR[i]
        V[i] = IR[i]*R 
        diff = abs(IL[1]-IL[0])
        ILguess[0] = ILguess[1]

    IL[i] = ILguess[1]
    L[i] = (phi[i]-phi[i-1])/(IL[i]-IL[i-1])
    print(IL[i])


65.64335751258048


KeyboardInterrupt: 

In [ ]:
NuT = 10
Lc = 0.01
A = 1e-6
R = 1e-2
Hc = 1e3 #scaling
Mc=  1e5 #scaling

k = 1
mu_0 = 4*np.pi*1e-7

Time = 0.5e-4
dt = 0.2e-6
n = np.floor(Time/dt)
Ival = 3

sw_speed = 1e7 #number of switches in one second
mc_steps = sw_speed*dt

I = []
I.extend(Ival*np.ones(50))
I.extend(-Ival*np.ones(50))
I.extend(Ival*np.ones(50))
k=len(I)

VL = [0]*len(I)
IL = [0]*len(I)
IR = [0]*len(I)
V = [0]*len(I)
ILguess=[0,0]
phi = [0]*len(I)
L = [0]*len(I)
H = [0]*len(I)
B = [0]*len(I)
M = [0]*len(I)

def monte_carlo(H,lattice):
    for i in range(mc_steps):
        x = np.random.randint(0,Nx)
        y = np.random.randint(0,Ny)
        z = np.random.randint(0,Nz)

        
        